# 5. Find largest observations (R)

In [1]:
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
pipelines_2010_selected_file = "../preprocessed_data/pipelines_2010_selected_2019-08-21.feather"

## Load data

In [3]:
pipelines_2010 <- feather::read_feather(pipelines_2010_selected_file)
glimpse(pipelines_2010)

Warning message:
“Coercing int64 to double”

Observations: 5,294
Variables: 19
$ OPERATOR_ID       <chr> "300", "300", "300", "395", "402", "473", "473", "5…
$ YEAR              <dbl> 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 201…
$ NAME              <chr> "PLAINS PIPELINE, L.P.", "PLAINS PIPELINE, L.P.", "…
$ COMMODITY         <chr> "Crude Oil", "HVL", "Refined and/or Petroleum Produ…
$ MILES             <dbl> 2101.060, 72.900, 0.000, 662.300, 1.550, 12.000, 0.…
$ AGE_UNKNOWN_MILES <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.…
$ PRE_1940_MILES    <dbl> 57.450, 0.000, 3.720, 0.600, 0.000, 0.000, 0.000, 0…
$ `1940_MILES`      <dbl> 739.300, 0.000, 0.000, 15.000, 0.000, 0.000, 0.000,…
$ `1950_MILES`      <dbl> 356.710, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, …
$ `1960_MILES`      <dbl> 647.640, 0.000, 0.000, 0.900, 0.000, 43.000, 0.000,…
$ `1970_MILES`      <dbl> 270.270, 0.000, 0.000, 1.500, 0.000, 4.000, 0.000, …
$ `1980_MILES`      <dbl> 93.110, 0.000, 0.000, 18.500, 0.000, 0.000, 2.000, …
$ `1990_MILES`    

## 5.1 Find largest observations

Ensure that we don't remove too many observations.

In [4]:
length(unique(pipelines_2010$OPERATOR_ID))

[1] 699

In [5]:
pipelines_2010 %>%
    group_by(OPERATOR_ID) %>%
    mutate(max_offshore = max(PERC_OFFSHORE)) %>%
    filter(max_offshore == 0) %>%
    ungroup() %>%
    {length(unique(.$OPERATOR_ID))}

[1] 653

Although it is not the exact number I expected, it is approximately correct.

In [6]:
pipelines_2010 %>%
    group_by(OPERATOR_ID) %>%
    mutate(max_offshore = max(PERC_OFFSHORE)) %>%
    filter(max_offshore == 0) %>%
    top_n(1, MILES) %>%
    ungroup() %>%
    select(OPERATOR_ID, YEAR, NAME, COMMODITY, MILES) %>%
    top_n(10, MILES) %>%
    arrange(desc(MILES))

OPERATOR_ID,YEAR,NAME,COMMODITY,MILES
<chr>,<dbl>,<chr>,<chr>,<dbl>
32109,2018,"ONEOK NGL PIPELINE, LLC",HVL,4531.48
2552,2015,COLONIAL PIPELINE CO,Refined and/or Petroleum Product (non-HVL),4500.92
1845,2015,"BUCKEYE PARTNERS, LP",Refined and/or Petroleum Product (non-HVL),4266.43
22610,2018,"MAGELLAN PIPELINE COMPANY, LP",Refined and/or Petroleum Product (non-HVL),3926.40
15674,2016,PLANTATION PIPE LINE CO,Refined and/or Petroleum Product (non-HVL),2564.73
11169,2017,"ENBRIDGE ENERGY, LIMITED PARTNERSHIP",Crude Oil,2290.00
31684,2010,CONOCOPHILLIPS,Refined and/or Petroleum Product (non-HVL),2283.79
18092,2014,"SFPP, LP",Refined and/or Petroleum Product (non-HVL),1786.46
10012,2018,NUSTAR PIPELINE OPERATING PARTNERSHIP L.P.,HVL,1781.60


#### Largest for the sum of all types

In [7]:
unique(pipelines_2010$COMMODITY)

[1] "Crude Oil"                                 
[2] "HVL"                                       
[3] "Refined and/or Petroleum Product (non-HVL)"
[4] "CO2"                                       
[5] "Fuel Grade Ethanol (dedicated system)"

Check whether any operator ID has more than the maximum of five expected entries.

In [8]:
pipelines_2010 %>%
    group_by(YEAR, OPERATOR_ID) %>%
    summarize(n = n()) %>%
    ungroup() %>%
    {table(.$n)}


   1    2    3    4 
3307  565  251   26 

Everything seems to be A-Okay.

In [9]:
largest_pipeline_operators <- pipelines_2010 %>%
    group_by(OPERATOR_ID) %>%
    mutate(max_offshore = max(PERC_OFFSHORE)) %>%
    filter(max_offshore == 0) %>%
    # Make sure we retrieve the latestb name, no matter what year has to most extensive pipeline network
    arrange(desc(YEAR)) %>%
    mutate(NAME = first(NAME)) %>%
    group_by(OPERATOR_ID, YEAR) %>%
    summarize(NAME = first(NAME), TOTAL_MILES = sum(MILES)) %>%
    group_by(OPERATOR_ID) %>%
    top_n(1, TOTAL_MILES) %>%
    # Year as tie breaker
    top_n(1, YEAR) %>%
    ungroup() %>%
    arrange(desc(TOTAL_MILES))

head(largest_pipeline_operators, 50)

OPERATOR_ID,YEAR,NAME,TOTAL_MILES
<chr>,<dbl>,<chr>,<dbl>
32109,2018,"ONEOK NGL PIPELINE, LLC",4756.610
22610,2014,"MAGELLAN PIPELINE COMPANY, LP",4505.500
2552,2015,COLONIAL PIPELINE CO,4500.920
31684,2017,PHILLIPS 66 PIPELINE LLC,4474.100
1845,2015,"BUCKEYE PARTNERS, LP",4298.290
18718,2010,SUNOCO PIPELINE L.P.,3466.740
22855,2013,"FLINT HILLS RESOURCES, LC",2757.450
10012,2018,NUSTAR PIPELINE OPERATING PARTNERSHIP L.P.,2645.300
15674,2016,PLANTATION PIPE LINE CO,2564.730


## 5.2 Save result

In [10]:
feather::write_feather(largest_pipeline_operators, paste0("../preprocessed_data/largest_companies_", Sys.Date(), ".feather"))